# Psi4 Tutorial 04: Characterization of stationary points with a harmonic vibrational analysis

In this tutorial you will learn how to determine if a stationary point obtained via geometry optimization is a local minimum or a transition state. We will do this by performing a vibrational analysis using the `psi4.frequency` command.

In [ ]:
import psi4

## Ammonia inversion

We will start with a very simple example. Let's consider the ammonia molecule and optimize its structure. Here we use a Z-matrix input to specify the geometry.

### Equilibrium geometry

First we start with the trigonal pyramidal, which is known to be the equilibrium geometry.
We start by performing a geometry optimization. Note that we save the psi4 `Molecule` object to the variable `eq`, so that we can refer to this later.

In this code the following line calls the `to_string` method of the `Molecule` object
```python
print(eq.to_string(dtype='xyz'))
```
to print the Cartesian coordinates of the geometry **before the optimization**.

In [ ]:
psi4.set_output_file("output.nh3.dat")

eq = psi4.geometry("""
0 1
X
N 1 1.0
H 2 R 1 A
H 2 R 1 A 3 D
H 2 R 1 A 3 -D

R = 1.000
A = 120.0
D = 120.0
""")

print(eq.to_string(dtype='xyz'))

psi4.optimize('scf/def2-SVP',molecule=eq)

To get the Cartesian coordinates of the **optimized geometry** we can call the `to_string` method again. This geometry is different from the one printed above, reflecting the changes that happen during the optimization process.

In [ ]:
print(eq.to_string(dtype='xyz'))

### Normal mode computation

To compute the Hessian and the corresponding vibrational frequencies, we call the `psi4.frequency` function. Before doing so we will set the psi4 option `NORMAL_MODES_WRITE` to `True` to tell psi4 to save the normal mode  information to a file:
```python
psi4.set_options({'NORMAL_MODES_WRITE' : True})
```

In [ ]:
# save the normal mode information
psi4.set_options({'NORMAL_MODES_WRITE' : True})

# run a frequency computation using the optimized geometry stored in the variable `eq`
e_eq, wfn_eq = psi4.frequency('scf/def2-SVP',molecule=eq,return_wfn=True)

From the wave function object (wfn) we can extract the vibrational frequencies (in cm$^{-1}$). Here we note that all frequencies are real, so **the optimized structure we found is a minimum**.

In [ ]:
# get the vibrational frequencies (in cm^-1) directly from the wavefunction object
freqs = wfn_eq.frequencies().to_array()
print(freqs)

The results of the harmonic vibrational analysis are printed at the bottom of the output file

In [ ]:
with open('output.nh3.dat','r') as f:
    lines = f.readlines()[-118:]
    print(''.join(lines))

### Normal mode visualization

To plot the normal modes you can use fortecubeview. The following lines allow us to read the normal modes generated in the frequency computation above. The syntax used here is a bit complicated because we have to compute the name of the file that stores this information (the name is different every time you run the frequency computation). If you plan on using this in your notebooks, just replace `eq` with the molecule object generated in your computation.

In [ ]:
import fortecubeview
filename = f'{psi4.core.get_writer_file_prefix(eq.name())}.molden_normal_modes'
fortecubeview.vib(filename)

### Planar geometry

Next, we will apply the vibrational analysis to the planar structure of ammonia (by setting the angle A to 90 degrees in the Z-matrix input).

In [ ]:
psi4.set_output_file("output.nh3_ts.dat")

ts = psi4.geometry("""
0 1
X
N 1 1.0
H 2 R 1 A
H 2 R 1 A 3 D
H 2 R 1 A 3 -D

R = 1.000
A = 90.0
D = 120.0
""")

psi4.set_options({'NORMAL_MODES_WRITE' : True})

print(ts.to_string(dtype='xyz'))

psi4.optimize('scf/def2-SVP',molecule=ts)

e_ts, wfn_ts = psi4.frequency('scf/def2-SVP',molecule=ts,return_wfn=True)

The output `Warning: thermodynamics relations excluded imaginary frequencies: ['908.6689i']` warns us that psi4 has detected one imaginary frequency (about $908.7 i$ cm<sup>-1</sup>). We can check the output file (recommended) or use code like the one below that prints the frequencies:

In [ ]:
freqs_ts = wfn_ts.frequencies().to_array()
print(freqs_ts)

with open('output.nh3_ts.dat','r') as f:
    lines = f.readlines()[-118:]
    print(''.join(lines))

Now we can take a look at the vibrational modes stored in the `ts` geometry object. We will use the `fortecubeview` module (which we already imported above).

In [ ]:
filename = f'{psi4.core.get_writer_file_prefix(ts.name())}.molden_normal_modes'
fortecubeview.vib(filename)

## The equilibrium structure of acetaldehyde

We will now move to show a simple application of the computation of frequencies that shows how important it can be to check if an optimized molecular structure corresponds to a minimum or not. We will optimize and compute the frequencies of acetaldehyde:

<img src="./c2h4o.png" alt="c2h4o molecule" width="150"/>

The following input is similar to the one used for ammonia.

In [ ]:
psi4.core.set_output_file('output.C2OH4.dat',False)

mol = psi4.geometry("""
C       -1.4851649825      1.0105098419     -0.0136919013                 
C       -1.4295256812     -0.4854850139      0.0016993677                 
O       -2.5145012116      1.6407920050     -0.2333702113                 
H       -0.4066169295     -0.8037578895      0.2192647845                 
H       -1.7209386966     -0.8720278934     -0.9774425038                 
H       -2.0934632091     -0.8677995447      0.7802096768                 
H       -0.5331793331      1.5302219067      0.1868246310                 
""")

psi4.set_options({'NORMAL_MODES_WRITE' : True})

psi4.optimize('scf/def2-SVP',molecule=mol)
e_ac1, wfn_ac1=psi4.frequencies('scf/def2-SVP',return_wfn=True,molecule=mol)

Surprisingly, we find that once we optimize the initial structure of acetaldehyde **we converge to a transition state geometry** (as indicated by the one imaginary frequency). Let's check the frequency analysis:

In [ ]:
freqs = wfn_ac1.frequencies().to_array()
print(freqs)

with open('output.C2OH4.dat','r') as f:
    lines = f.readlines()[-184:]
    print(''.join(lines))

What mode gives the imaginary frequency? If we visualize the normal modes we find that the imaginary mode corresponds to a rotation of the methyl group. Here the two hydrogen are eclypsed and this configuration is not a minimum.

In [ ]:
filename = f'{psi4.core.get_writer_file_prefix(mol.name())}.molden_normal_modes'
fortecubeview.vib(filename)

To find the minimum, we can rotate the C-C bond and reoptimize

In [ ]:
psi4.core.set_output_file('output.C2OH4.2.dat',False)

mol2 = psi4.geometry("""
C       -1.4851649825      1.0105098419     -0.0136919013                 
C       -1.4295256812     -0.4854850139      0.0016993677                 
O       -2.1398156566      1.6485701762     -0.8318353741                 
H       -0.4066169295     -0.8037578895      0.2192647845                 
H       -1.7209386966     -0.8720278934     -0.9774425038                 
H       -2.0934632091     -0.8677995447      0.7802096768                 
H       -0.8949834415      1.5227111606      0.7647165676                 
""")

psi4.optimize('scf/def2-SVP',molecule=mol2)
e_2b,wfn_2b=psi4.frequencies('scf/def2-SVP',return_wfn=True,molecule=mol2)

This frequency computaion does not show any imaginary frequency. The warning you may see (`Warning: used thermodynamics relations inappropriate for low-frequency modes: ['158.1215' '551.3462']`) just tells us that there are some low-energy frequencies.

In [ ]:
filename = f'{psi4.core.get_writer_file_prefix(mol2.name())}.molden_normal_modes'
fortecubeview.vib(filename)

## Plotting the IR absorption spectrum

Once you have performed a frequency computation, you can plot the IR spectrum of a molecule. This can be useful to compare your results to experimentally measured spectra.

The following python function plots the frequencies by reading the output of a psi4 computation. Just specify the name of the output file like in the following line of python
```python
plot_freqs('output.C2OH4.2.dat')  
```
When you combine this with the normal mode analysis in fortecubeview you can then use computation to assign the peaks in the IR spectrum of a molecule to a specific motion of the atoms.

In [ ]:
def plot_freqs(filename):
    """
    Function to plot the IR spectrum.

    Usage: plot_freqs(filename)

    Inputs: name of psi4 output file from SCF calculation
    filename: name

    Output: plot of the IR spectrum
    """

    import math
    import matplotlib.pyplot as plt

    with open(filename) as f:
        frequencies = []
        intensities = []        
        for line in f:
            if 'Freq [cm^-1]' in line:
                for val in [float(omega) for omega in line.split()[2:]]:
                    frequencies.append(val)
            if 'IR activ [km/mol]' in line:
                for val in [float(omega) for omega in line.split()[3:]]:
                    intensities.append(val)

    xmin = min(min(frequencies),0.0)
    xmax = max(frequencies) + 25.0
    npoints = 600
    alpha = 1.0 / 100.0
    dx = (xmax - xmin)/ float(npoints)
    xvals = [xmin + dx * i for i in range(npoints)]
    yvals = []
    for x in xvals:
        y = 0.0
        for f,i in zip(frequencies,intensities):
            y += i * math.exp(- alpha * (x - f)**2)
        yvals.append(y)

    plt.plot(xvals,yvals)
    plt.xlabel('wavelength (cm^-1)')
    plt.ylabel('intensity (km/mol)')
    plt.title('IR Spectrum')
    plt.show() 
    
plot_freqs('output.C2OH4.2.dat')   